# Proposed Method
- We proposed **DTML** for stock price prediction
    - *Data-axis Transformer with Multi-Level contexts* 
1. Idea 1_Time-axis attension
    - 8/11 가격을 예측하기 위해서 지난 여러 날짜의 데이터를 사용한다. (8/1~8/10)
    - 직관적으로 가장 가까운 날짜의 데이터가 영향을 줄 것으로 생각할 수 있다.
        - 하지만, 주식마다 다를 수 있다. 
        - 어떤 날짜의 데이터가 8/11 가격 예측에 도움이 되는지를 동적으로 attention 기법으로 학습
    - To summarize the historical prices of each stock
2. Idea 2_Context aggregation
    - 주식은 시장과 독립될 수 없다.
    - 개별 주식의 트렌드 + 시장의 트렌드
    - To combine individual contexts with a global trend
        - 개별 종목을 예측할 때 글로벌 인덱스도 같이 고려하자는 의미 
3. Idea 3_Data-axis attention
    - 여러 종목 간의 상관관계를 파악해야 함. 사전에 고정된 정보를 주지 말고, 동적으로 학습을 하자는 것
    - To learn the stock correlations by a transformer
            - 주식 종목 간의 연관관계를 transformer를 이용해서 학습함

# Model Architecture_1
- Three modules correspond to the three main ideas.
1. Time-Axis Attention
    - 개별 종목 : ex. 삼성전자, 하이닉스, NC
    - 각 종목의 가격들이 매일 들어감 (RNN)
    - 각 종목의 지난 날짜의 가격에 대한 데이터를 얻음
    - market index가 매일 들어가서 데이터를 얻음
        - KOSPI의 매일 가격이 들어감
2. Context Aggregation  
    - 개별 종목의 벡터와 market context를 더해서 Multi-Level Context를 얻게 됨
    - Multi-Level Context : 개별 종목의 정보 뿐만 아니라 전체 마켓의 정보까지 포함
3. Data-Axis Attention 
    - 여러 개의 종목에 대한 벡터를 얻게 됨
    - 이 벡터들에 대한 가중평균을 구한다.
    - transformer 모델을 사용
    - 개별적으로 각 종목에 대한 벡터를 얻고, Linear layer를 통해서 실제로 오를지 내릴지를 binary classification 진행

# Model Architecture_2
1. Module 1. Time-axis attention
    - Summarizes the historical prices as a single vector.
        - 개별 종목의 가격을 하나의 벡터로 요약함
        - 각각의 가격 벡터가 Feature Transformation Layer를 통해서 새로운 벡터로 주어짐
        -   LSTM의 입력으로 들어감
        - 주식 u가 있을 때 지난 날짜 T까지의 벡터 세트가 있다고 한다면, 
        -   MLD(?) layer를 통과해서 feature 변환을 하게 됨
    - Attention LSTM
        - LSTM
            - x1, x2, x3 -> 맨 마지막 hidden vector를 일반적으로 summary vector로 사용한다. (== context vector)
        - Attention LSTM
            - context vector를 h1, h2, h3의 가중 평균으로 구한다. 
            - h3만 사용하는 것이 아니라 지난 모든 hidden vector의 가중 평균을 구하자
                - 그러면, 가중치는 어떻게 구할까?
                - 맨 마지막 hidden vector와 각 벡터의 내적의 softmax 형태로 비례하는 식으로 weight를 정해준다.
            - 결과적으로 각 종목의 vector를 가중평균으로 구하는데, 가중치는 벡터의 내적에 비례하는 식으로 구한다.
2. Module 2. Context normalization
    - 각 종목의 가격이 굉장히 다양한 scale로 있기 때문에 normalize를 진행해야 한다.
    - Time-Axis Attention을 거치면 각 종목의 벡터를 얻게 된다.
        - Market Context (KOSPI vector)
    - market과 개별 주식을 더한다. 
        - 주식 u가 있을 때 개별 종목의 context vector와 마켓 인덱스를 하이퍼 파라미터를 넣으면서 더하게 된다. 
        - global market context (ex. KOSPI, KOSDAQ, NASDAQ)
        - context를 더하는 게 어떤 효과를 줄까?
            - context를 더한 후에 종목u와 종목v의 벡터의 내적을 구하게 되면, 
            -   (원래 종목의 내적) + (마켓 인덱스 * 각 종목의 연관관계) + (마켓 인덱스 자체에 의한 기본적인 correlation)
            -       (마켓 인덱스 * 각 종목의 연관관계) : 연관성이 높으면 높은 correlation을 가지게 됨

3. Module 3. Data-axis attention
    - 각 종목별 vector가 있는데, 이 vector들의 가중평균을 다시 구함 <= transformer를 사용해서 각 종목의 가중평균을 구함
    - 종목의 개수가 n이고, 각 종목별로 v개의 feature를 가지고 있다면 => H matrix
    -   H matrix에 가중